In [1]:
from __future__ import annotations

from typing import Callable

import pytest
import torch

import pyqtorch as pyq
from pyqtorch import DiffMode, expectation
from pyqtorch.analog import Observable
from pyqtorch.circuit import QuantumCircuit
from pyqtorch.matrices import COMPLEX_TO_REAL_DTYPES
from pyqtorch.parametric import Parametric
from pyqtorch.utils import GPSR_ACCEPTANCE, PSR_ACCEPTANCE, GRADCHECK_sampling_ATOL

PyQTorch logger successfully setup with log level 20


In [2]:
import random
from pyqtorch.analog import Add, Scale
from pyqtorch.apply import apply_operator
from pyqtorch.circuit import Sequence
from pyqtorch.parametric import (
    OPS_PARAM_1Q,
    OPS_PARAM_2Q,
    Parametric,
)
from pyqtorch.primitive import (
    OPS_1Q,
    OPS_2Q,
    OPS_3Q,
    OPS_PAULI,
    OPS_PAULI_with_generator,
    Primitive,
    Toffoli,
)

def random_pauli_hamiltonian(
    n_qubits: int,
    k_1q: int = 5,
    k_2q: int = 10,
    make_param: bool = False,
    exclude_N: bool = False,
) -> tuple[Sequence, list]:
    """Creates a random Pauli Hamiltonian as a sum of k_1q + k_2q terms."""
    OPS_PAULI_choice = list(OPS_PAULI)
    if exclude_N:
        OPS_PAULI_choice = list(OPS_PAULI_with_generator)
    one_q_terms: list = random.choices(OPS_PAULI_choice, k=k_1q)
    two_q_terms: list = [random.choices(OPS_PAULI_choice, k=2) for _ in range(k_2q)]
    terms: list = []
    for term in one_q_terms:
        supp = random.sample(range(n_qubits), 1)
        terms.append(term(supp))
    for term in two_q_terms:
        supp = random.sample(range(n_qubits), 2)
        terms.append(Sequence([term[0](supp[0]), term[1](supp[1])]))
    param_list = []
    for i, t in enumerate(terms):
        if random.random() > 0.5:
            if make_param:
                terms[i] = Scale(t, f"p_{i}")
                param_list.append(f"p_{i}")
            else:
                terms[i] = Scale(t, torch.rand(1))
    return Add(terms), param_list

In [3]:
def circuit_gpsr(n_qubits: int) -> QuantumCircuit:
    """Helper function to make an example circuit using multi gap GPSR."""

    ops = [
        pyq.Y(1),
        pyq.RX(0, "theta_0"),
        pyq.PHASE(0, "theta_1"),
        pyq.CSWAP(0, (1, 2)),
        pyq.CRX(1, 2, "theta_2"),
        pyq.CPHASE(1, 2, "theta_3"),
        pyq.CNOT(0, 1),
        #pyq.Toffoli((0, 1), 2),
    ]

    circ = QuantumCircuit(n_qubits, ops)

    return circ

In [4]:
n_qubits = 5
dtype = torch.complex128
batch_size = 3

In [5]:
 
obs2 = Observable(pyq.Add([
    pyq.Y(4),
    pyq.Scale(pyq.Y(3), torch.tensor(0.8823 + 0.0j)),
    pyq.Y(4),
    pyq.Scale(pyq.Z(1), torch.tensor(0.9150 + 0.0j)),
    pyq.Scale(pyq.Y(2), torch.tensor(0.3829 + 0.0j)),
    pyq.Sequence([pyq.Z(4),pyq.X(3),]),
    pyq.Scale(pyq.Sequence([pyq.X(0),pyq.Y(4)]), torch.tensor(0.9513 + 0.0j)),
    pyq.Sequence([pyq.X(1),pyq.X(4),]),
    pyq.Scale(pyq.Sequence([pyq.X(4),pyq.Z(1)]), torch.tensor(0.3904 + 0.0j)),
    pyq.Scale(pyq.Sequence([pyq.Z(0),pyq.Z(3)]), torch.tensor(0.6009 + 0.0j)),
    
])).to(dtype)

/Users/charles.moussa/Library/Application Support/hatch/env/virtual/pyqtorch/Gy8xvlfd/pyqtorch/lib/python3.10/site-packages/torch/nn/modules/module.py:1144: UserWarning: Complex modules are a new feature under active development whose design may change, and some modules might not work as expected when using complex tensors as parameters or buffers. Please file an issue at https://github.com/pytorch/pytorch/issues/new?template=bug-report.yml if a complex module does not work as expected.
  warnings.warn(


In [22]:
torch.manual_seed(42)
circ = circuit_gpsr(n_qubits).to(dtype)

values = {
    op.param_name: torch.rand(
        batch_size, requires_grad=True, dtype=COMPLEX_TO_REAL_DTYPES[dtype]
    )
    for op in circ.flatten()
    if isinstance(op, Parametric) and isinstance(op.param_name, str)
}

In [23]:
values

{'theta_0': tensor([0.0582, 0.0629, 0.1236], dtype=torch.float64, requires_grad=True),
 'theta_1': tensor([0.0526, 0.5262, 0.4768], dtype=torch.float64, requires_grad=True),
 'theta_2': tensor([0.9552, 0.9288, 0.0835], dtype=torch.float64, requires_grad=True),
 'theta_3': tensor([0.1326, 0.1571, 0.3754], dtype=torch.float64, requires_grad=True)}

In [24]:
state = pyq.random_state(n_qubits, dtype=dtype)

In [32]:


# Apply adjoint
exp_ad = expectation(circ, state, values, obs2, DiffMode.AD)
grad_ad = torch.autograd.grad(
    exp_ad, tuple(values.values()), torch.ones_like(exp_ad), create_graph=True
)

# Apply PSR
exp_gpsr = expectation(circ, state, values, obs2, DiffMode.GPSR)
grad_gpsr = torch.autograd.grad(
    exp_gpsr, tuple(values.values()), torch.ones_like(exp_gpsr), create_graph=True
)

exp_gpsr_sampled = expectation(
    circ,
    state,
    values,
    obs2,
    DiffMode.GPSR,
    n_shots=100000,
)
grad_gpsr_sampled = torch.autograd.grad(
    exp_gpsr_sampled,
    tuple(values.values()),
    torch.ones_like(exp_gpsr_sampled),
    create_graph=True,
)
assert torch.allclose(exp_gpsr, exp_gpsr_sampled, atol=1e-01)

atol = 1.0e-05

# first order checks

for i in range(len(grad_ad)):
    assert torch.allclose(grad_ad[i], grad_gpsr[i], atol=atol)
    assert torch.allclose(
        grad_ad[i], grad_gpsr_sampled[i], atol=GRADCHECK_sampling_ATOL
    )

In [33]:
exp_ad, exp_gpsr, exp_gpsr_sampled

(tensor([-0.5767, -0.6116, -0.7841], dtype=torch.float64,
        grad_fn=<SelectBackward0>),
 tensor([-0.5767, -0.6116, -0.7841], dtype=torch.float64,
        grad_fn=<PSRExpectationBackward>),
 tensor([-0.5946, -0.6123, -0.7743], dtype=torch.float64,
        grad_fn=<PSRExpectationBackward>))

In [34]:
grad_ad, grad_gpsr_sampled

((tensor([0.2755, 0.2619, 0.3279], dtype=torch.float64, grad_fn=<AddBackward0>),
  tensor([-0.0713, -0.0417, -0.0978], dtype=torch.float64,
         grad_fn=<SelectBackward0>),
  tensor([0.1615, 0.1911, 0.1711], dtype=torch.float64, grad_fn=<AddBackward0>),
  tensor([-0.1700, -0.1699, -0.1076], dtype=torch.float64,
         grad_fn=<SelectBackward0>)),
 (tensor([0.2827, 0.2601, 0.3312], dtype=torch.float64, grad_fn=<ViewBackward0>),
  tensor([-0.0632, -0.0340, -0.1070], dtype=torch.float64,
         grad_fn=<ViewBackward0>),
  tensor([0.1462, 0.1751, 0.1985], dtype=torch.float64, grad_fn=<MulBackward0>),
  tensor([-0.1695, -0.1677, -0.1084], dtype=torch.float64,
         grad_fn=<ViewBackward0>)))